In [5]:
!pip install opencv-python
!pip install opencv-python mediapipe

   ---------------------------------------- 0.0/39.5 MB ? eta -:--:--
   ---------------------------------------- 0.0/39.5 MB ? eta -:--:--
   ---------------------------------------- 0.0/39.5 MB ? eta -:--:--
   ---------------------------------------- 0.3/39.5 MB ? eta -:--:--
   ---------------------------------------- 0.3/39.5 MB ? eta -:--:--
    --------------------------------------- 0.5/39.5 MB 670.4 kB/s eta 0:00:59
    --------------------------------------- 0.8/39.5 MB 699.0 kB/s eta 0:00:56
    --------------------------------------- 0.8/39.5 MB 699.0 kB/s eta 0:00:56
   - -------------------------------------- 1.0/39.5 MB 729.5 kB/s eta 0:00:53
   - -------------------------------------- 1.3/39.5 MB 762.6 kB/s eta 0:00:51
   - -------------------------------------- 1.3/39.5 MB 762.6 kB/s eta 0:00:51
   - -------------------------------------- 1.6/39.5 MB 798.7 kB/s eta 0:00:48
   - -------------------------------------- 1.8/39.5 MB 831.8 kB/s eta 0:00:46
   -- ------------


[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


   ---------------------------------------- 0.0/51.0 MB ? eta -:--:--
   ---------------------------------------- 0.3/51.0 MB ? eta -:--:--
   ---------------------------------------- 0.5/51.0 MB 1.4 MB/s eta 0:00:37
    --------------------------------------- 0.8/51.0 MB 1.4 MB/s eta 0:00:36
    --------------------------------------- 1.0/51.0 MB 1.4 MB/s eta 0:00:35
   - -------------------------------------- 1.3/51.0 MB 1.5 MB/s eta 0:00:35
   - -------------------------------------- 1.8/51.0 MB 1.5 MB/s eta 0:00:32
   - -------------------------------------- 2.1/51.0 MB 1.6 MB/s eta 0:00:32
   -- ------------------------------------- 2.6/51.0 MB 1.6 MB/s eta 0:00:30
   -- ------------------------------------- 2.9/51.0 MB 1.6 MB/s eta 0:00:30
   -- ------------------------------------- 3.4/51.0 MB 1.7 MB/s eta 0:00:29
   -- ------------------------------------- 3.7/51.0 MB 1.7 MB/s eta 0:00:29
   --- ------------------------------------ 3.9/51.0 MB 1.7 MB/s eta 0:00:28
   --- ------


[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import cv2
import mediapipe as mp
import numpy as np
import winsound  # ✅ For sound alert on Windows

# --- EAR and MAR calculation ---
def calculate_ear(landmarks, eye_indices):
    p = [np.array([landmarks[i].x, landmarks[i].y]) for i in eye_indices]
    A = np.linalg.norm(p[1] - p[5])
    B = np.linalg.norm(p[2] - p[4])
    C = np.linalg.norm(p[0] - p[3])
    ear = (A + B) / (2.0 * C)
    return ear

def calculate_mar(landmarks):
    # Use only stable top and bottom lip center points
    top_lip = np.array([landmarks[13].x, landmarks[13].y])
    bottom_lip = np.array([landmarks[14].x, landmarks[14].y])
    vertical = np.linalg.norm(top_lip - bottom_lip)

    # Use left and right corner of the mouth
    left = np.array([landmarks[61].x, landmarks[61].y])
    right = np.array([landmarks[291].x, landmarks[291].y])
    horizontal = np.linalg.norm(left - right)

    mar = vertical / horizontal
    return mar


# --- Constants ---
EAR_THRESHOLD = 0.25
MAR_THRESHOLD = 0.7
EAR_CONSEC_FRAMES = 20
COUNTER = 0

# --- Landmark indexes ---
LEFT_EYE = [362, 385, 387, 263, 373, 380]
RIGHT_EYE = [33, 160, 158, 133, 153, 144]
MOUTH = [61, 81, 13, 311, 308, 402, 14, 178, 88, 95, 78, 191, 80, 81, 82, 13, 312]

# --- Setup Mediapipe ---
mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(static_image_mode=False, max_num_faces=1, refine_landmarks=True)
mp_drawing = mp.solutions.drawing_utils

# --- Start video capture ---
cap = cv2.VideoCapture(0)
print("⏳ Starting webcam. Press 'q' to quit.")

while cap.isOpened():
    success, frame = cap.read()
    if not success:
        print("❌ Failed to capture frame.")
        break

    h, w = frame.shape[:2]
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = face_mesh.process(rgb_frame)

    if results.multi_face_landmarks:
        for face_landmarks in results.multi_face_landmarks:
            landmarks = face_landmarks.landmark

            ear_left = calculate_ear(landmarks, LEFT_EYE)
            ear_right = calculate_ear(landmarks, RIGHT_EYE)
            ear_avg = (ear_left + ear_right) / 2.0

            mar = calculate_mar(landmarks)

            # === Face Box ===
            x_coords = [int(lm.x * w) for lm in landmarks]
            y_coords = [int(lm.y * h) for lm in landmarks]
            x_min, x_max = min(x_coords), max(x_coords)
            y_min, y_max = min(y_coords), max(y_coords)
            padding = 30
            x_min = max(0, x_min - padding)
            y_min = max(0, y_min - padding)
            x_max = min(w, x_max + padding)
            y_max = min(h, y_max + padding)
            cv2.rectangle(frame, (x_min, y_min), (x_max, y_max), (0, 255, 0), 2)

            # === Display EAR and MAR values ===
            cv2.putText(frame, f"EAR: {ear_avg:.2f}", (30, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (100, 255, 100), 2)
            cv2.putText(frame, f"MAR: {mar:.2f}", (30, 60), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (100, 200, 255), 2)

            # === Drowsiness detection ===
            alert = False
            if ear_avg < EAR_THRESHOLD:
                COUNTER += 1
                if COUNTER >= EAR_CONSEC_FRAMES:
                    alert = True
                    cv2.putText(frame, "⚠️ DROWSY!", (200, 100), cv2.FONT_HERSHEY_SIMPLEX, 1.2, (0, 0, 255), 3)
            else:
                COUNTER = 0

            if mar > MAR_THRESHOLD:
                alert = True
                cv2.putText(frame, "😮 YAWNING!!!", (200, 140), cv2.FONT_HERSHEY_SIMPLEX, 1.0, (0, 255, 255), 3)

            # === Sound alert if drowsy ===
            if alert:
                winsound.Beep(1000, 300)  # Frequency, Duration in ms

    cv2.imshow("Drowsy Driver Detection", frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

⏳ Starting webcam. Press 'q' to quit.
